In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
from PIL import Image

# 配置Matplotlib以便在Jupyter Notebook中显示图像
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 设置工作目录
base_dir = r'F:/train'  # 训练数据的根目录
# 定义输出目录
output_dir = r"F:/train_pre"
classification_label_file = os.path.join(base_dir, "train_classification_label.xlsx")

# 定义图像处理参数
image_size = (224, 224)  # 图像大小 (宽, 高)
batch_size = 32  # 批处理大小
random_seed = 41572  # 随机种子

# 设置随机种子以确保结果可复现
np.random.seed(random_seed)
random.seed(random_seed)

# 加载和读取标签数据
使用pandas读取train_classification_label.xlsx文件，分析标签分布，并探索数据集的基本结构。

In [ ]:
# 使用pandas读取标签文件
labels_df = pd.read_excel(classification_label_file)

# 查看标签数据的前几行
print("标签数据的前几行：")
print(labels_df.head())

# 检查数据集的基本信息
print("\n数据集基本信息：")
print(labels_df.info())

# 检查标签分布
print("\n标签分布：")
label_distribution = labels_df['Pterygium'].value_counts()
print(label_distribution)

# 可视化标签分布
plt.figure(figsize=(6, 4))
label_distribution.plot(kind='bar', color=['green', 'orange', 'red'])
plt.title("标签分布")
plt.xlabel("类别")
plt.ylabel("数量")
plt.xticks(ticks=[0, 1, 2], labels=["健康", "建议观察", "建议手术"], rotation=0)
plt.show()

# 图像加载和预览
创建图像加载函数，从train文件夹中读取图像，并显示健康、建议观察和建议手术三种类别的样例图像。

In [ ]:
# 定义一个函数用于加载图像
def load_image(image_path):
    """
    加载图像并调整大小为指定尺寸。
    :param image_path: 图像文件路径
    :return: 调整大小后的图像
    """
    image = Image.open(image_path).convert("RGB")  # 打开图像并转换为RGB模式
    image = image.resize(image_size)  # 调整图像大小
    return np.array(image)

# 从每个类别中随机选择一个样例图像进行预览
def preview_sample_images(base_dir, labels_df):
    """
    从健康、建议观察和建议手术类别中各选择一个样例图像并显示。
    :param base_dir: 图像根目录
    :param labels_df: 包含图像标签的DataFrame
    """
    categories = {0: "健康", 1: "建议观察", 2: "建议手术"}
    plt.figure(figsize=(12, 4))
    
    for category, label_name in categories.items():
        # 获取当前类别的所有图像
        category_images = labels_df[labels_df['Pterygium'] == category]['Image'].values
        if len(category_images) == 0:
            print(f"类别 {label_name} 没有图像。")
            continue
        
        # 随机选择一个图像
        sample_image_id = random.choice(category_images)
        # 确保图像ID是字符串格式，并且格式化为4位数
        sample_image_name = f"{int(sample_image_id):04d}"
        sample_image_path = os.path.join(base_dir, sample_image_name, f"{sample_image_name}.png")
        
        # 加载并显示图像
        sample_image = load_image(sample_image_path)
        plt.subplot(1, 3, category + 1)
        plt.imshow(sample_image)
        plt.title(label_name)
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

# 调用函数预览样例图像
preview_sample_images(base_dir, labels_df)

# 图像标准化处理
实现图像标准化处理，包括大小调整、归一化、对比度增强等操作，确保所有图像具有一致的特征。

In [ ]:
# 定义图像标准化处理函数
def preprocess_image(image):
    """
    对输入图像进行标准化处理，包括大小调整、归一化和对比度增强。
    :param image: 输入图像 (numpy array)
    :return: 标准化后的图像
    """
    # 转换为浮点数并归一化到 [0, 1]
    image = image.astype(np.float32) / 255.0
    
    # 对比度增强 (使用直方图均衡化)
    image_uint8 = (image * 255).astype(np.uint8)
    image_ycrcb = cv2.cvtColor(image_uint8, cv2.COLOR_RGB2YCrCb)
    y, cr, cb = cv2.split(image_ycrcb)
    y = cv2.equalizeHist(y)
    image_enhanced = cv2.merge([y, cr, cb])
    image_enhanced = cv2.cvtColor(image_enhanced, cv2.COLOR_YCrCb2RGB)
    
    # 转回浮点数格式
    image_enhanced = image_enhanced.astype(np.float32) / 255.0
    
    return image_enhanced

# 定义高光消除函数
def remove_highlights(image):
    """
    消除图像中的高光/反光
    :param image: 输入图像 (numpy array)
    :return: 去除高光后的图像
    """
    # 将图像转为 uint8 格式处理
    img_uint8 = (image * 255).astype(np.uint8)
    
    # 转换为 LAB 色彩空间，L通道对亮度更敏感
    lab = cv2.cvtColor(img_uint8, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    
    # 创建高光掩码 - 使用自适应阈值找出亮度异常高的区域
    # 使用CLAHE改善对比度而不是简单阈值
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    
    # 检测高亮区域
    _, highlight_mask = cv2.threshold(l, 200, 255, cv2.THRESH_BINARY)
    
    # 细化掩码
    kernel = np.ones((3, 3), np.uint8)
    highlight_mask = cv2.dilate(highlight_mask, kernel, iterations=1)
    highlight_mask = cv2.GaussianBlur(highlight_mask, (5, 5), 0)
    
    # 生成三通道掩码并归一化到 [0, 1]
    highlight_mask_3d = cv2.merge([highlight_mask, highlight_mask, highlight_mask]) / 255.0
    
    # 使用引导滤波进行修复
    img_inpaint = cv2.inpaint(img_uint8, highlight_mask, 5, cv2.INPAINT_TELEA)
    
    # 只在高光区域应用修复，其他区域保留原始图像
    result = img_uint8 * (1 - highlight_mask_3d) + img_inpaint * highlight_mask_3d
    
    # 返回归一化结果
    return result.astype(np.float32) / 255.0

# 综合预处理函数
def complete_preprocessing(image):
    """
    完整的图像预处理流程：标准化和高光消除
    :param image: 输入图像 (numpy array)
    :return: 处理后的图像
    """
    # 首先标准化
    normalized = preprocess_image(image)
    
    # 然后消除高光
    no_highlights = remove_highlights(normalized)
    
    return no_highlights

# 测试图像预处理
def test_preprocessing(base_dir, labels_df):
    """
    测试图像预处理并显示处理效果
    :param base_dir: 图像根目录
    :param labels_df: 包含图像标签的DataFrame
    """
    # 随机选择一个图像进行测试
    sample_image_id = labels_df['Image'].iloc[100]
    sample_image_name = f"{int(sample_image_id):04d}"
    sample_image_path = os.path.join(base_dir, sample_image_name, f"{sample_image_name}.png")
    
    # 加载原始图像
    original_image = load_image(sample_image_path)
    
    # 应用各种预处理
    normalized_image = preprocess_image(original_image)
    no_highlights_image = remove_highlights(original_image)
    complete_image = complete_preprocessing(original_image)
    
    # 显示处理前后的对比
    plt.figure(figsize=(15, 4))
    
    plt.subplot(1, 4, 1)
    plt.imshow(original_image)
    plt.title("原始图像")
    plt.axis("off")
    
    plt.subplot(1, 4, 2)
    plt.imshow(normalized_image)
    plt.title("标准化")
    plt.axis("off")
    
    plt.subplot(1, 4, 3)
    plt.imshow(no_highlights_image)
    plt.title("高光消除")
    plt.axis("off")
    
    plt.subplot(1, 4, 4)
    plt.imshow(complete_image)
    plt.title("完整处理")
    plt.axis("off")
    
    plt.tight_layout()
    plt.show()

test_preprocessing(base_dir, labels_df)

# 处理完整数据集
创建一个流水线，批量处理整个数据集中的图像，并将处理后的图像保存到新目录中，为后续模型训练做准备。

In [ ]:
# 定义处理完整数据集的函数
def process_dataset(base_dir, labels_df, output_dir):
    """
    批量处理整个数据集中的图像，包括标准化和高光消除，并将处理后的图像保存到新目录中。
    :param base_dir: 图像根目录
    :param labels_df: 包含图像标签的DataFrame
    :param output_dir: 处理后图像的保存目录
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for _, row in labels_df.iterrows():
        image_name = row['Image']
        image_label = row['Pterygium']
        image_name = f"{int(image_name):04d}"
        input_image_path = os.path.join(base_dir, image_name, image_name + ".png")
        
        # 加载图像
        if not os.path.exists(input_image_path):
            print(f"图像 {input_image_path} 不存在，跳过...")
            continue
        image = load_image(input_image_path)
        
        # 图像标准化
        normalized_image = preprocess_image(image)
        
        # 高光消除
        processed_image = remove_specular_highlight(normalized_image)
        
        # 保存处理后的图像
        output_label_dir = os.path.join(output_dir, f"class_{image_label}")
        if not os.path.exists(output_label_dir):
            os.makedirs(output_label_dir)
        output_image_path = os.path.join(output_label_dir, image_name + ".png")
        Image.fromarray((processed_image * 255).astype(np.uint8)).save(output_image_path)
    
    print(f"数据集处理完成，处理后的图像保存在 {output_dir}")

# 调用函数处理数据集
# process_dataset(base_dir, labels_df, output_dir)